In [8]:
import pandas as pd
license = pd.read_csv('Licensed_Marijuana_Businesses_in_Colorado.csv')
license.head()


C:\Users\james\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Licensee,DBA,License_No,Street_Address,City,ZIP,Category,Month,Year,Certification
0,1617 WAZEE STREET LLC,NaN,NaN,1617 Wazee St,Denver,80202.0,Medical Marijuana Centers,1,2013,NaN
1,3-D DENVER'S DISCREET DISPENSARY LLC,NaN,NaN,4305 Brighton Blvd,Brighton,80216.0,Medical Marijuana Centers,1,2013,NaN
2,"4625 E. COLFAX, LLC",NaN,NaN,4625 E. Colfax Ave,Denver,80216.0,Medical Marijuana Centers,1,2013,NaN
3,5280 WELLNESS LLC,NaN,NaN,1321 Elati Street,Denver,80204.0,Medical Marijuana Centers,1,2013,NaN
4,A CUT OFF THE TOP MEDICAL MARIJUANA,NaN,NaN,2059 W. 9th Ave,Denver,80204.0,Medical Marijuana Centers,1,2013,NaN


In [9]:
sales_revenue=pd.read_csv('Marijuana_Sales_Revenue_in_Colorado.csv')
sales_revenue.head()

,Month,Year,County,Med_Sales,Rec_Sales,Med_Blank_Code,Rec_Blank_Code,ID
0,12,2019,Yuma,NaN,NaN,NL,NL,122019Yuma
1,12,2019,Weld,314840.0,2448689.0,NaN,NaN,122019Weld
2,12,2019,Washington,NaN,NaN,NL,NL,122019Washington
3,12,2019,Teller,NaN,NaN,NL,NL,122019Teller
4,12,2019,Summit,NaN,2476854.0,NR,NaN,122019Summit


In [10]:
med_mj_stats=pd.read_csv('Medical_Marijuana_Statistics_-_County_Information.csv')
med_mj_stats.head()

,County,Number of Patients,Percent of Patients
0,Adams,8772,8.0
1,Alamosa,204,NaN
2,Arapahoe,11985,11.0
3,Archuleta,378,NaN
4,Baca,35,NaN


In [12]:
med_mj_stats.dtypes

County                  object
Number of Patients       int64
Percent of Patients    float64
dtype: object

In [11]:
tax_revenue=pd.read_csv('State_Sales_Tax_Revenue_from_Marijuana_in_Colorado.csv')
tax_revenue.head()

,Month,Year,County,Med_Tax_Rev,Rec_Tax_Rev,Med_RemainderOfState_Counties,Rec_RemainderOfState_Counties,Med_Blank_Code,Rec_Blank_Code,ID
0,8,2019,Adams,17037.0,NaN,NaN,NaN,NaN,NaN,82019Adams
1,8,2019,Alamosa,NaN,NaN,NaN,NaN,NR,NaN,82019Alamosa
2,8,2019,Arapahoe,24256.0,NaN,NaN,NaN,NaN,NaN,82019Arapahoe
3,8,2019,Boulder,33557.0,NaN,NaN,NaN,NaN,NaN,82019Boulder
4,8,2019,Chaffee,NaN,NaN,NaN,NaN,NR,NaN,82019Chaffee
